In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
import warnings
import itertools
warnings.filterwarnings("ignore")
import arviz as az
import scipy
import matplotlib.patches as patches
import statsmodels.stats.api as sms
import scipy.stats.kde as kde

# Read 16S data

In [2]:
df_16S = pd.read_excel('./16S_relative_abundance.xlsx', index_col=0)
df_16S.columns = [c.replace('/','_slash_').replace(' ','_space_').replace('[','_leftsquarebracket_').replace(']','_rightsquarebracket_').replace('-','_dash_').replace('.','_dot_').replace('(','_leftroundbracket').replace(')','_rightroundbracket_') for c in df_16S.columns]
df_16S.head()

,Acetanaerobacterium,Acidaminococcus_dash_intestini,Akkermansia,Akkermansia_dash_sp_dot__dash_KLE1798,Alistipes,Alistipes_dash_finegoldii_dash_DSM_dash_17242,Alistipes_dash_inops,Alistipes_dash_obesi,Alistipes_dash_shahii_dash_WAL_dash_8301,Alistipes_dash_sp_dot__dash_Marseille_dash_P5997,...,bacterium_dash_enrichment_dash_culture_dash_clone_dash_RB7c,butyrate_dash_producing_dash_bacterium_dash_L2_dash_10,gut_dash_metagenome,human_dash_gut_dash_metagenome,iron_dash_reducing_dash_bacterium_dash_enrichment_dash_culture_dash_clone_dash_HN70,metagenome,methanogenic_dash_archaeon_dash_CH1270,rumen_dash_bacterium_dash_NK2A1,rumen_dash_bacterium_dash_YS2,vadinBE97
SampleID,,,,,,,,,,,,,,,,,,,,,
sample1,0.0,0.0,0.012382,0.0,0.018176,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.005000,0.000873,0.0,0.005715,0.0,0.0,0.0,0.0
sample2,0.0,0.0,0.000000,0.0,0.011868,0.003349,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.008914,0.002265,0.0,0.012607,0.0,0.0,0.0,0.0
sample3,0.0,0.0,0.030136,0.0,0.022406,0.008317,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.011350,0.002642,0.0,0.019520,0.0,0.0,0.0,0.0
sample4,0.0,0.0,0.003595,0.0,0.045532,0.005492,0.0,0.002097,0.0,0.0,...,0.0,0.0,0.011183,0.002546,0.0,0.011083,0.0,0.0,0.0,0.0
sample5,0.0,0.0,0.005820,0.0,0.008059,0.000000,0.0,0.002361,0.0,0.0,...,0.0,0.0,0.045299,0.004396,0.0,0.008628,0.0,0.0,0.0,0.0


# Add pseudocount for zeros

In [3]:
for sample_id in df_16S.index:
    sample = np.array(df_16S.loc[sample_id])
    minval = np.min(sample[np.nonzero(sample)]) # minimum non-zero value
    sample[sample==0] = minval
    df_16S.loc[sample_id] = sample
df_16S.head()

,Acetanaerobacterium,Acidaminococcus_dash_intestini,Akkermansia,Akkermansia_dash_sp_dot__dash_KLE1798,Alistipes,Alistipes_dash_finegoldii_dash_DSM_dash_17242,Alistipes_dash_inops,Alistipes_dash_obesi,Alistipes_dash_shahii_dash_WAL_dash_8301,Alistipes_dash_sp_dot__dash_Marseille_dash_P5997,...,bacterium_dash_enrichment_dash_culture_dash_clone_dash_RB7c,butyrate_dash_producing_dash_bacterium_dash_L2_dash_10,gut_dash_metagenome,human_dash_gut_dash_metagenome,iron_dash_reducing_dash_bacterium_dash_enrichment_dash_culture_dash_clone_dash_HN70,metagenome,methanogenic_dash_archaeon_dash_CH1270,rumen_dash_bacterium_dash_NK2A1,rumen_dash_bacterium_dash_YS2,vadinBE97
SampleID,,,,,,,,,,,,,,,,,,,,,
sample1,0.000635,0.000635,0.012382,0.000635,0.018176,0.000635,0.000635,0.000635,0.000635,0.000635,...,0.000635,0.000635,0.005000,0.000873,0.000635,0.005715,0.000635,0.000635,0.000635,0.000635
sample2,0.000148,0.000148,0.000148,0.000148,0.011868,0.003349,0.000148,0.000148,0.000148,0.000148,...,0.000148,0.000148,0.008914,0.002265,0.000148,0.012607,0.000148,0.000148,0.000148,0.000148
sample3,0.000196,0.000196,0.030136,0.000196,0.022406,0.008317,0.000196,0.000196,0.000196,0.000196,...,0.000196,0.000196,0.011350,0.002642,0.000196,0.019520,0.000196,0.000196,0.000196,0.000196
sample4,0.000449,0.000449,0.003595,0.000449,0.045532,0.005492,0.000449,0.002097,0.000449,0.000449,...,0.000449,0.000449,0.011183,0.002546,0.000449,0.011083,0.000449,0.000449,0.000449,0.000449
sample5,0.000244,0.000244,0.005820,0.000244,0.008059,0.000244,0.000244,0.002361,0.000244,0.000244,...,0.000244,0.000244,0.045299,0.004396,0.000244,0.008628,0.000244,0.000244,0.000244,0.000244


# Read metadata for maize starch and control

In [18]:
set(df_meta.treatmentgroup)

{'inulin-maltodextrin', 'maltodextrin-inulin'}

In [56]:
df_meta = pd.read_csv('./metadata-filtered.txt', sep='\t', index_col=0)
df_meta = df_meta.drop('#q2:types')
df_meta = df_meta[df_meta.excluded=='FALSE']
df_meta = df_meta[['subject_id','inulin','timepoint','treatmentgroup']]

# remove patients that have odd number of samples
subject_to_remove = []
for sid in set(df_meta.subject_id):
    _df = df_meta[df_meta.subject_id==sid]
    if len(_df) % 2 != 0:
        subject_to_remove.append(sid)
df_meta = df_meta[~df_meta.subject_id.isin(subject_to_remove)]

# runin1->intake1, runin2->intake2
for sid in set(df_meta.subject_id):
    df_meta.loc[(df_meta.subject_id==sid)&((df_meta.timepoint=='RunIn1')|(df_meta.timepoint=='Intake1')),'subject_id'] = sid+'_A'
    df_meta.loc[(df_meta.subject_id==sid)&((df_meta.timepoint=='RunIn2')|(df_meta.timepoint=='Intake2')),'subject_id'] = sid+'_B'
    
for sid in set(df_meta.subject_id):
    df_meta.loc[(df_meta.subject_id==sid)&((df_meta.timepoint=='RunIn1')|(df_meta.timepoint=='RunIn2')),'timepoint'] = 0
    df_meta.loc[(df_meta.subject_id==sid)&((df_meta.timepoint=='Intake1')|(df_meta.timepoint=='Intake2')),'timepoint'] = 21
    
for sid in set(df_meta.subject_id):
    treatment = list(set(list(df_meta.loc[df_meta.subject_id==sid,'treatmentgroup'])))
    assert len(treatment)==1
    if sid.endswith('_A'):
        if treatment[0].split('-')[0]=='inulin':
            df_meta.loc[df_meta.subject_id==sid,'inulin']=True
        else:
            df_meta.loc[df_meta.subject_id==sid,'inulin']=False
    else:
        # ends with _B
        if treatment[0].split('-')[1]=='inulin':
            df_meta.loc[df_meta.subject_id==sid,'inulin']=True
        else:
            df_meta.loc[df_meta.subject_id==sid,'inulin']=False

df_meta.timepoint = df_meta.timepoint.astype(int)
df_meta = df_meta.rename({'timepoint':'Day', 'subject_id':'Subject', 'inulin':'Diet'}, axis=1)
df_meta['Diet'] = df_meta['Diet'].astype(int)
df_meta = df_meta.drop('treatmentgroup', axis=1)
df_meta.head()

,Subject,Diet,Day
#SampleID,,,
sample1,1_A,1,21
sample2,1_B,0,21
sample3,1_A,1,0
sample4,1_B,0,0
sample5,3_A,0,21


# Select the 20 most abundant taxa

In [57]:
topX = 20
df_16S_T = df_16S.loc[df_meta.index].T
df_16S_T['mean'] = df_16S_T.mean(axis=1)
df_16S_T = df_16S_T.sort_values(by=['mean'],axis=0,ascending=False)
df_16S_T = df_16S_T.drop('mean', axis=1)
df_16S_topX = df_16S_T.iloc[0:topX].T
df_16S_topX.head()

,Bacteroides,Faecalibacterium,Lachnospiraceae,gut_dash_metagenome,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Alistipes,Blautia,Coprococcus,metagenome,Anaerostipes,Dorea,Bifidobacterium,human_dash_gut_dash_metagenome,Roseburia,Christensenellaceae_dash_R_dash_7_dash_group,Ruminococcus,Subdoligranulum,Fusicatenibacter,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_hallii_dash_group,Ruminococcus_dash_sp_dot__dash_DJF_VR52
#SampleID,,,,,,,,,,,,,,,,,,,,
sample1,0.016827,0.044130,0.059370,0.005000,0.064767,0.018176,0.011429,0.006429,0.005715,0.008334,0.009207,0.023018,0.000873,0.000635,0.003651,0.048734,0.019287,0.000635,0.013334,0.013811
sample2,0.038018,0.071161,0.062395,0.008914,0.029203,0.011868,0.015710,0.016104,0.012607,0.002512,0.013641,0.003989,0.002265,0.019403,0.006008,0.022161,0.029105,0.008027,0.011376,0.021078
sample3,0.094369,0.054645,0.090015,0.011350,0.033805,0.022406,0.013453,0.012915,0.019520,0.006800,0.020009,0.018003,0.002642,0.000196,0.006458,0.045546,0.031114,0.002984,0.018933,0.019764
sample4,0.137344,0.043784,0.059910,0.011183,0.062157,0.045532,0.016525,0.010634,0.011083,0.008238,0.019970,0.006291,0.002546,0.000449,0.009685,0.072691,0.022866,0.003145,0.020170,0.008088
sample5,0.172894,0.112617,0.038665,0.045299,0.017135,0.008059,0.013675,0.072527,0.008628,0.011152,0.008384,0.000244,0.004396,0.000244,0.075824,0.005698,0.008506,0.011559,0.006553,0.006349


In [58]:
list(df_16S_topX.columns)

['Bacteroides',
 'Faecalibacterium',
 'Lachnospiraceae',
 'gut_dash_metagenome',
 'Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57',
 'Alistipes',
 'Blautia',
 'Coprococcus',
 'metagenome',
 'Anaerostipes',
 'Dorea',
 'Bifidobacterium',
 'human_dash_gut_dash_metagenome',
 'Roseburia',
 'Christensenellaceae_dash_R_dash_7_dash_group',
 'Ruminococcus',
 'Subdoligranulum',
 'Fusicatenibacter',
 '_leftsquarebracket_Eubacterium_rightsquarebracket__dash_hallii_dash_group',
 'Ruminococcus_dash_sp_dot__dash_DJF_VR52']

# Normalize max value of 16S data to 1

In [59]:
bacterial_taxa = list(df_16S_topX.columns)
df_16S_topX_w_meta = pd.merge(df_meta, df_16S_topX/df_16S_topX.max().max(), left_index=True, right_index=True, how='inner')
df_16S_topX_w_meta.head()

,Subject,Diet,Day,Bacteroides,Faecalibacterium,Lachnospiraceae,gut_dash_metagenome,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Alistipes,Blautia,...,Dorea,Bifidobacterium,human_dash_gut_dash_metagenome,Roseburia,Christensenellaceae_dash_R_dash_7_dash_group,Ruminococcus,Subdoligranulum,Fusicatenibacter,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_hallii_dash_group,Ruminococcus_dash_sp_dot__dash_DJF_VR52
#SampleID,,,,,,,,,,,,,,,,,,,,,
sample1,1_A,1,21,0.024823,0.065101,0.087582,0.007377,0.095544,0.026813,0.016861,...,0.013582,0.033956,0.001288,0.000937,0.005386,0.071892,0.028453,0.000937,0.019671,0.020373
sample2,1_B,0,21,0.056085,0.104977,0.092046,0.013149,0.043081,0.017508,0.023175,...,0.020124,0.005885,0.003342,0.028623,0.008863,0.032692,0.042935,0.011842,0.016782,0.031094
sample3,1_A,1,0,0.139213,0.080612,0.132790,0.016743,0.049869,0.033053,0.019846,...,0.029517,0.026558,0.003897,0.000289,0.009526,0.067189,0.045899,0.004402,0.027929,0.029156
sample4,1_B,0,0,0.202610,0.064591,0.088379,0.016497,0.091694,0.067168,0.024378,...,0.029460,0.009280,0.003756,0.000663,0.014288,0.107234,0.033731,0.004640,0.029754,0.011931
sample5,3_A,0,21,0.255053,0.166133,0.057039,0.066825,0.025277,0.011888,0.020174,...,0.012368,0.000360,0.006484,0.000360,0.111856,0.008406,0.012548,0.017052,0.009667,0.009366


# Calculate log-derivatives (dlog(B)/dt)

In [60]:
df_deriv = deepcopy(df_16S_topX_w_meta)
for curr_sub in set(df_deriv.Subject):
    curr_df = df_deriv[df_deriv.Subject==curr_sub].sort_values(by='Day')
    for taxon in bacterial_taxa:
        xdata = np.array(curr_df['Day'])
        ydata = np.array(curr_df[taxon])
        cs = CubicSpline(xdata, ydata)
        csd1 = cs.derivative(nu=1)
        ydata_d1 = csd1(xdata)
        df_deriv.loc[df_deriv.Subject==curr_sub, taxon] = ydata_d1
df_deriv.head()

,Subject,Diet,Day,Bacteroides,Faecalibacterium,Lachnospiraceae,gut_dash_metagenome,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Alistipes,Blautia,...,Dorea,Bifidobacterium,human_dash_gut_dash_metagenome,Roseburia,Christensenellaceae_dash_R_dash_7_dash_group,Ruminococcus,Subdoligranulum,Fusicatenibacter,_leftsquarebracket_Eubacterium_rightsquarebracket__dash_hallii_dash_group,Ruminococcus_dash_sp_dot__dash_DJF_VR52
#SampleID,,,,,,,,,,,,,,,,,,,,,
sample1,1_A,1,21,-0.005447,-0.000739,-0.002153,-0.000446,0.002175,-0.000297,-0.000142,...,-0.000759,0.000352,-0.000124,0.000031,-0.000197,0.000224,-0.000831,-0.000165,-0.000393,-0.000418
sample2,1_B,0,21,-0.006977,0.001923,0.000175,-0.000159,-0.002315,-0.002365,-0.000057,...,-0.000445,-0.000162,-0.000020,0.001331,-0.000258,-0.003550,0.000438,0.000343,-0.000618,0.000912
sample3,1_A,1,0,-0.005447,-0.000739,-0.002153,-0.000446,0.002175,-0.000297,-0.000142,...,-0.000759,0.000352,-0.000124,0.000031,-0.000197,0.000224,-0.000831,-0.000165,-0.000393,-0.000418
sample4,1_B,0,0,-0.006977,0.001923,0.000175,-0.000159,-0.002315,-0.002365,-0.000057,...,-0.000445,-0.000162,-0.000020,0.001331,-0.000258,-0.003550,0.000438,0.000343,-0.000618,0.000912
sample5,3_A,0,21,0.000615,-0.000337,-0.000245,0.000335,-0.001594,0.000134,0.000260,...,-0.000269,-0.000233,-0.000536,0.000011,0.001363,0.000268,-0.000352,-0.000302,-0.000610,-0.000948


# Construct regression matrix

In [61]:
# Ymat should be samples by bacteria
Ymat = df_deriv[bacterial_taxa].values
Ymat = Ymat.flatten(order='F')
Ymat = StandardScaler().fit_transform(Ymat.reshape(-1,1)).reshape(1,-1)[0] # standardize
Ymat

array([-2.38964576, -3.05821695, -2.38964576, ...,  0.09548217,
       -0.15072508,  0.09548217])

In [62]:
len(Ymat)

3240

In [63]:
Xmat = np.zeros(shape=(topX*len(df_deriv.index), (topX+2)*topX))
for k in np.arange(topX):
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)] = 1
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+1] = df_deriv.Diet.values
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+2:(k+1)*(topX+2)] = df_16S_topX_w_meta[bacterial_taxa].values
Xmat

array([[1.        , 1.        , 0.02482278, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.05608459, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 0.1392133 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.02161112, 0.00993983,
        0.01975141],
       [0.        , 0.        , 0.        , ..., 0.01140879, 0.01369055,
        0.01538074],
       [0.        , 0.        , 0.        , ..., 0.03487752, 0.00555013,
        0.01469523]])

# Write stan program

## write data

In [68]:
json_str = '{\n"N" : %d,\n'%(len(Ymat))

# y variable
json_str += '\"dlogX\" : [%s],\n'%(','.join(list(Ymat.astype(str))))

# x variable
for k1,c1 in enumerate(bacterial_taxa):
    # growth rate
    json_str += '\"growth_rate_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)].astype(str))))
    # diet response
    json_str += '\"rs_response_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)+1].astype(str))))
    # bacterial interactions
    for k2,c2 in enumerate(bacterial_taxa):
        v = list(Xmat[:,k1*(topX+2)+2+k2].astype(str))
        json_str += '\"pairwise_interaction_%s_%s\" : [%s]'%(c1,c2,','.join(v))
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            json_str += '\n}'
        else:
            json_str += ',\n'
#print(json_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.data.json", "w")
text_file.write("%s" % json_str)
text_file.close()

## write model

In [69]:
# data block
model_str = 'data {\n'
model_str += '\tint<lower=0> N;\n'
model_str += '\tvector[N] dlogX;\n'
for c1 in bacterial_taxa:
    model_str += '\tvector[N] growth_rate_%s;\n'%(c1)
    model_str += '\tvector[N] rs_response_%s;\n'%(c1)
    for c2 in bacterial_taxa:
        model_str += '\tvector[N] pairwise_interaction_%s_%s;\n'%(c1,c2)
model_str += '}\n'

# parameter block
model_str += 'parameters {\n\treal<lower=0,upper=1> sigma;\n'
for c1 in bacterial_taxa:
    model_str += '\treal alpha__%s;\n'%(c1) # growth rate
    model_str += '\treal epsilon__%s;\n'%(c1) # rs response
    for c2 in bacterial_taxa:
        model_str += '\treal beta__%s_%s;\n'%(c1,c2)
model_str += '}\n'       
        
# model block
model_str += 'model {\n\tsigma ~ uniform(0,1);\n'
for c1 in bacterial_taxa:
    model_str += '\talpha__%s ~ normal(0,1);\n'%(c1) # growth rate
    model_str += '\tepsilon__%s ~ normal(0,1);\n'%(c1) # rs response
    for c2 in bacterial_taxa:
        model_str += '\tbeta__%s_%s ~ normal(0,1);\n'%(c1,c2)
model_str += '\tdlogX ~ normal('
for c1 in bacterial_taxa:
    model_str += 'alpha__%s*growth_rate_%s+'%(c1,c1) # growth rate
    model_str += 'epsilon__%s*rs_response_%s+'%(c1,c1) # rs response
    for c2 in bacterial_taxa:
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s'%(c1,c2,c1,c2)
        else:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s+'%(c1,c2,c1,c2)
model_str += ', sigma);\n}'
#print(model_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.stan", "w")
text_file.write("%s" % model_str)
text_file.close()

# Plot stan output

In [70]:
def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
        x_hpd = x[(x > value[0]) & (x < value[1])]
        y_hpd = y[(x > value[0]) & (x < value[1])]
        modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes

In [71]:
fit = az.from_cmdstan(["output_%d.csv"%(i) for i in np.arange(1,4)])

In [72]:
lines = []
    
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
# inulin response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['rs_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])

# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.head()

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Bacteroides,-0.42,0.50,0.040,0.159749,False
1,basal_growth_rate,Faecalibacterium,-0.60,0.32,-0.140,-0.605190,False
2,basal_growth_rate,Lachnospiraceae,-0.66,0.26,-0.200,-0.856932,False
3,basal_growth_rate,gut_dash_metagenome,-0.14,0.78,0.320,1.414249,False
4,basal_growth_rate,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_das...,-0.34,0.57,0.115,0.486693,False


In [73]:
df_stan_output_summary.to_excel('bayesian_regression_summary_top20.xlsx')

# Plot

In [74]:
df_stan_output_summary = pd.read_excel('bayesian_regression_summary_top20.xlsx', index_col=0)
df_stan_output_summary = df_stan_output_summary[df_stan_output_summary.Significant==True]
df_stan_output_summary

,Type,Taxa,Left,Right,Middle,SNR,Significant
22,rs_response,Lachnospiraceae,0.11,0.70,0.405,2.684754,True
24,rs_response,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_das...,-0.64,-0.05,-0.345,-2.313357,True
29,rs_response,Anaerostipes,0.10,0.70,0.400,2.642284,True
31,rs_response,Bifidobacterium,0.19,0.78,0.485,3.229673,True
40,pairwise_interaction,"('Bacteroides', 'Bacteroides')",0.54,2.00,1.270,3.432835,True
41,pairwise_interaction,"('Bacteroides', 'Faecalibacterium')",-7.65,-4.91,-6.280,-9.042536,True
43,pairwise_interaction,"('Bacteroides', 'gut_dash_metagenome')",0.02,3.46,1.740,1.989138,True
44,pairwise_interaction,"('Bacteroides', 'Ruminococcus_dash_sp_dot__das...",0.51,4.05,2.280,2.536649,True
46,pairwise_interaction,"('Bacteroides', 'Blautia')",-3.70,-0.08,-1.890,-2.059112,True
61,pairwise_interaction,"('Faecalibacterium', 'Faecalibacterium')",1.53,4.27,2.900,4.187724,True
